In [28]:
from exif import Image
from datetime import datetime

In [29]:

def get_time(image):
    with open(image, 'rb') as image_file:
        img = Image(image_file)
        time_str = img.get("datetime_original")
        time = datetime.strptime(time_str, '%Y:%m:%d %H:%M:%S')
    return time


In [30]:
print(get_time("imgs\photo_0676.jpg"))

2023-05-08 15:30:56


In [31]:
def get_time_difference(image_1, image_2):
    time_1 = get_time(image_1)
    time_2 = get_time(image_2)
    time_difference = time_2 - time_1
    return time_difference.seconds

In [32]:
print(get_time_difference('imgs/photo_1754.jpg', 'imgs/photo_1755.jpg'))

6


In [33]:
from exif import Image
from datetime import datetime
import cv2
import math

In [34]:
def convert_to_cv(image_1, image_2):
    image_1_cv = cv2.imread(image_1, 0)
    image_2_cv = cv2.imread(image_2, 0)
    return image_1_cv, image_2_cv

In [35]:
def calculate_features(image_1, image_2, feature_number):
    orb = cv2.ORB_create(nfeatures = feature_number)
    keypoints_1, descriptors_1 = orb.detectAndCompute(image_1_cv, None)
    keypoints_2, descriptors_2 = orb.detectAndCompute(image_2_cv, None)
    return keypoints_1, keypoints_2, descriptors_1, descriptors_2

In [70]:

#@ filtrovani features tak aby jsme ignorovali mraky(byli features)
def filter_features(keypoints_1, keypoints_2, descriptors_1, descriptors_2):
    """
    zjisti svetlost kazdeho keypointu, vyfiltruje keypointy ktere jsou moc svetle(pravdepodobne mraky)

    args: 
    keypoints(tupple): output funkce calculate_features

    returns:
    filtered_features(tupple): features ktere by nemeli byt mraky, nejsou bile
    """
    #*nejaka moje predstava postupu - rozhodne to neni jedina ani "spravna" cesta jak to vyresit ale vymyslel jsem to tak proc to sem nedat zejo
    # definuju nejakej threshold svetlosti(0-255) - pokud to bude svetlajsi jak threshold, je to asi mrak
    # muzu proiterovat nekolika temahle hodnotama a zjistit si kdy my to ignoruje vse a kdy to zas neignoruje nic
    # pak vyberu tu hodnotu kdyb to zhruba ignoruje vsechny mraky ale porad zbyvaji jine features
    
    # zjistim x,y souradnice keypointu
    # zjistim svetlost pixelu na obrazku na techto souradnicich
    # pokud je svetlost mensi(min svetla) nez muj threshold, keypoint zaradim do tupplu fiktered_features
    # opakuju pro vsechny keypoity
    # return filtered_keypoints
    print(keypoints_1[1])
    #* abych zjistil jakej data threshold
    #* proiteruju skrz keypointy, zjistim jejich svetlost   
    for kp_1 in keypoints_1:
        svetlost_1 = image_1_cv[int(kp_1.pt[1]), int(kp_1.pt[0])]
        print(f"svetlost kp_1: {svetlost_1}")
    return keypoints_1, keypoints_2, descriptors_1, descriptors_2

In [37]:
def calculate_matches(descriptors_1, descriptors_2):
    brute_force = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = brute_force.match(descriptors_1, descriptors_2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

In [38]:
def display_matches(image_1_cv, keypoints_1, image_2_cv, keypoints_2, matches):
    match_img = cv2.drawMatches(image_1_cv, keypoints_1, image_2_cv, keypoints_2, matches[:100], None)
    resize = cv2.resize(match_img, (1600,600), interpolation = cv2.INTER_AREA)
    cv2.imshow('matches', resize)
    #@ zavrit okno klavesou "0"
    cv2.waitKey(0)
    cv2.destroyWindow('matches')

In [39]:
#@ tqdm se musime zbavit pred odevzdanim
from tqdm import tqdm
def find_matching_coordinates(keypoints_1, keypoints_2, matches):
    coordinates_1 = []
    coordinates_2 = []
    for match in tqdm(matches):
        image_1_idx = match.queryIdx
        image_2_idx = match.trainIdx
        (x1,y1) = keypoints_1[image_1_idx].pt
        (x2,y2) = keypoints_2[image_2_idx].pt
        coordinates_1.append((x1,y1))
        coordinates_2.append((x2,y2))
    return coordinates_1, coordinates_2

In [40]:
def calculate_mean_distance(coordinates_1, coordinates_2):
    all_distances = 0
    merged_coordinates = list(zip(coordinates_1, coordinates_2))
    for coordinate in merged_coordinates:
        x_difference = coordinate[0][0] - coordinate[1][0]
        y_difference = coordinate[0][1] - coordinate[1][1]
        distance = math.hypot(x_difference, y_difference)
        all_distances = all_distances + distance
    return all_distances / len(merged_coordinates)

In [41]:
def calculate_distance_traveled(distance_on_earth):
    """
    zohledni vliv zakrivevni zeme na vdalenost urazenou ISS oproti vzdalenosti na zemi

    args:
    distance_on_earth(float): vysledek funkce calculate_mean_distance

    returns:
    distance_traveled(float)
    """
    # tady proste ta matika na spocitani jaka bude vzdalenost urazena ISS
    # zname:
    # vzdalenost kterou "urazil" bod na zemi,
    # zhruba polomer "drahy" toho bodu
    #zhruba polomer drahy ISS, polomer zeme + obehova vyska ISS
    #
    # o tomhle ví mnohem víc Denys
    # ale jestli jsem to v patek dobre pochopil tak by tohle melo stacit
    
    
    pass

In [42]:
def calculate_speed_in_kmps(feature_distance, GSD, time_difference):
    distance = (feature_distance * GSD)/ 100000
    speed = distance / time_difference
    return speed

In [43]:
image_1 = "imgs/photo_0676.jpg"
image_2 = "imgs/photo_0677.jpg"

In [71]:
time_difference = get_time_difference(image_1, image_2) # Get time difference between images
image_1_cv, image_2_cv = convert_to_cv(image_1, image_2) # Create OpenCV image objects
keypoints_1, keypoints_2, descriptors_1, descriptors_2 = calculate_features(image_1_cv, image_2_cv, 1000) # Get keypoints and descriptors
keypoints_1, keypoints_2, descriptors_1, descriptors_2 = filter_features(keypoints_1,keypoints_2, descriptors_1, descriptors_2) #* filtered keypoints and descriptors
matches = calculate_matches(descriptors_1, descriptors_2) # Match descriptors
#@display_matches(image_1_cv, keypoints_1, image_2_cv, keypoints_2, matches) # Display matches
coordinates_1, coordinates_2 = find_matching_coordinates(keypoints_1, keypoints_2, matches)
average_feature_distance = calculate_mean_distance(coordinates_1, coordinates_2)
speed = calculate_speed_in_kmps(average_feature_distance, 12648, time_difference)
print(f"prumerna vzdalenost features (km) - {average_feature_distance}")
print(f"rozdil casu (s) - {time_difference}")
print(f"rychlost (km/s) - {speed}")

< cv2.KeyPoint 0000020C24D91BC0>
svetlost kp_1: 140
svetlost kp_1: 206
svetlost kp_1: 221
svetlost kp_1: 129
svetlost kp_1: 158
svetlost kp_1: 158
svetlost kp_1: 177
svetlost kp_1: 182
svetlost kp_1: 228
svetlost kp_1: 160
svetlost kp_1: 149
svetlost kp_1: 126
svetlost kp_1: 165
svetlost kp_1: 185
svetlost kp_1: 202
svetlost kp_1: 203
svetlost kp_1: 138
svetlost kp_1: 198
svetlost kp_1: 190
svetlost kp_1: 189
svetlost kp_1: 183
svetlost kp_1: 163
svetlost kp_1: 123
svetlost kp_1: 202
svetlost kp_1: 169
svetlost kp_1: 215
svetlost kp_1: 201
svetlost kp_1: 152
svetlost kp_1: 195
svetlost kp_1: 192
svetlost kp_1: 196
svetlost kp_1: 210
svetlost kp_1: 194
svetlost kp_1: 166
svetlost kp_1: 152
svetlost kp_1: 202
svetlost kp_1: 188
svetlost kp_1: 182
svetlost kp_1: 130
svetlost kp_1: 182
svetlost kp_1: 173
svetlost kp_1: 193
svetlost kp_1: 158
svetlost kp_1: 190
svetlost kp_1: 189
svetlost kp_1: 159
svetlost kp_1: 198
svetlost kp_1: 174
svetlost kp_1: 179
svetlost kp_1: 185
svetlost kp_1: 18

100%|██████████| 511/511 [00:00<?, ?it/s]

prumerna vzdalenost features (km) - 515.572672946997
rozdil casu (s) - 9
rychlost (km/s) - 7.2455146304817974
